# [dyskinesia project] Preprocessing Neurophysiology Data


This notebooks helps to Step-by-Step select and preprocess ECoG and LFP (STN electrodes) data within the ReTune-Dyskinesia project.
<b> Data is required to converted into the BIDS-standard. </b>

For automated preprocessing, check: XXXX.py

### 0. Loading packages and functions, defining paths



In [2]:
# Importing Python and external packages
import os
import sys
import importlib
import json
from abc import ABCMeta, abstractmethod
from dataclasses import dataclass, field, fields
from collections import namedtuple
from typing import Any
from itertools import compress
from pathlib import Path
import pandas as pd
import numpy as np
import sklearn as sk
import scipy
import matplotlib.pyplot as plt
from scipy import signal

#mne
import mne_bids
import mne

In [3]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('mne_bids', mne_bids.__version__)
print('mne', mne.__version__)
print('sci-py', scipy.__version__)
print('sci-kit learn', sk.__version__)

Python sys 3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]
pandas 1.3.4
numpy 1.20.3
mne_bids 0.9
mne 0.24.1
sci-py 1.7.1
sci-kit learn 1.0.1


In [4]:
# define local storage directories
projectpath = '/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys'
datapath = os.path.join(projectpath, 'data/BIDS_Berlin_ECOG_LFP/rawdata')
codepath = os.path.join(projectpath, 'code')
figpath = os.path.join(projectpath, 'figures')
pynmd_path = os.path.join(codepath, 'py_neuromodulation')

# define external storage directories
ext_projectpath = '/Volumes/JH/Research/CHARITE/projects/dyskinesia_neurophys'
ext_datapath = os.path.join(ext_projectpath, 'data/BIDS_Berlin_ECOG_LFP/rawdata')


In [91]:
# import from py_neuromodulation after setting directory
os.chdir(pynmd_path)
print(os.getcwd())

import dyskinesia.preprocessing as preproc
import dyskinesia.reref as reref
import dyskinesia.preproc_artefacts as artefacts
import dyskinesia.preproc_filters as fltrs


/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys/code/py_neuromodulation


### 1. Data selection, defining Settings

- First DataClass defines which run/data-file should be used (via RunInfo)
- Second DataClass creates MNE-objects ordered by data-type (via RunRawData) 


Note that the resulting Data-Class Objects do not contain the actual data yet (!)
- Create RawBrainVision data-objects: load data with rawRun1.ecog.load_data() (incl. internal mne-functionality)
- Create np.array's: load data with rawRun1.ecog.get_data(), use return_times=True to return two tuples (data, times); (used in preprocessing.py functions)

BIDS-RAW Data Structure Info:
- Grouped MNE BIDS Raw Object consists all channels within the group,
e.g. lfp_left, lfp_left, ecog, acc. Each channel (rawRun1.ecog[0])
is a tuple with the first object a ndarray of shape 1, N_samples.
- Calling rawRun1.ecog[0][0] gives the ndarray containing only data-points.
- Calling rawRun1.ecog[1] gives the ndarray containing the time stamps.


#### Preprocess Settings definition


TODO: Create .py + .json scripts for automated pipeline


In [33]:
# Define structures (namedtuples) to store Default settings
PreprocSettings = namedtuple('PreprocSettings', (
    'win_len '
    'artfct_sd_tresh '
    'bandpass_f '
    'transBW '
    'notchW '
    'Fs_orig '
    'Fs_resample '
    'settings_f_name '
    'fig_path '))
Settings = namedtuple('Settings', 'lfp_left lfp_right ecog')

In [38]:
default_lfp_settings = [1, 2.5, (1, 120), 10, 2, 4000, 800, 'v0.1 Jan22', figpath]
default_ecog_settings = [1, 2.5, (1, 120), 10, 2, 4000, 800, 'v0.1 Jan22', figpath]

In [39]:
settings = Settings(
    PreprocSettings(*default_lfp_settings),
    PreprocSettings(*default_lfp_settings),
    PreprocSettings(*default_ecog_settings),
# '*' before lists unpacks the list-values as seperate args
)
groups = list(settings._fields)

#### Patient and Recording information


TODO: Create .py + .json scripts for automated pipeline


In [36]:
# DEFINE PTATIENT-RUN SETTINGS
sub = '008'
ses = 'EphysMedOn02'
task = 'Rest'
acq = 'StimOffLD00'
run = '01'
sourcepath = ext_datapath

In [40]:
# create specific patient-run BIDS-Object for further pre-processing
importlib.reload(preproc)
runInfo0 = preproc.RunInfo(
    sub=sub,
    ses=ses,
    task=task,
    acq=acq,
    run=run,
    sourcepath=sourcepath,
    preproc_sett=settings.lfp_left.settings_f_name,
    fig_path=figpath,
)
rawRun = preproc.RunRawData(bidspath=runInfo0.bidspath)



------------ BIDS DATA INFO ------------
The raw-bids-object contains 47 channels with 2444946 datapoints and sample freq  4000.0 Hz
Bad channels are: ['LFP_L_16_STN_BS'] 

BIDS contains:
6 ECOG channels,
31 DBS channels: (15 left, 16 right), 
2 EMG channels, 
1 ECG channel(s), 
6 Accelerometry (misc) channels.




/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys/code/py_neuromodulation/dyskinesia/preprocessing.py:89: RuntimeWarning: Did not find any events.tsv associated with sub-008_ses-EphysMedOn02_task-Rest_acq-StimOffLD00_run-01.

The search_str was "/Volumes/JH/Research/CHARITE/projects/dyskinesia_neurophys/data/BIDS_Berlin_ECOG_LFP/rawdata/sub-008/**/ieeg/sub-008_ses-EphysMedOn02*events.tsv"
  self.acc = self.bids.copy().pick_types(misc=True, exclude='bads')
/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys/code/py_neuromodulation/dyskinesia/preprocessing.py:89: RuntimeWarning: Defaulting coordinate frame to unknown from coordinate system input Other
  self.acc = self.bids.copy().pick_types(misc=True, exclude='bads')
/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys/code/py_neuromodulation/dyskinesia/preprocessing.py:89: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  self.acc = self.bid

#### Optional viewer for un-processed data with MNE's interactive viewer

In [68]:
# to load grouped BIDS-Objects:
# rawRun1.ecog.load_data()

# to visualize non-pre-processed data PSD's
# for interactive plotter: activate matplotlib qt line
# %matplotlib qt
# %matplotlib inline

# rawRun1.lfp_left.plot()
# rawRun1.lfp_left.plot_psd(n_fft=1024)
# rawRun1.ecog.plot_psd(n_fft=1024)

### 2. Automated Artefact Removal (incl. Visualization)


In [41]:
# Actual Loading of the Data from BIDS-files

# data_raw is filled with loaded mne-bids data per group
data_raw = {}
for field in rawRun.__dataclass_fields__:
    print(field)
    # loops over variables within the data class
    if str(field)[:4] == 'lfp_':
        data_raw[str(field)] = getattr(rawRun, field).load_data()
    elif str(field)[:4] == 'ecog':
        data_raw[str(field)] = getattr(rawRun, field).load_data()

ch_names = {}
for group in groups:
    ch_names[group] = data_raw[group].info['ch_names']

bidspath
bids
lfp
lfp_left
Reading 0 ... 2444945  =      0.000 ...   611.236 secs...
lfp_right
Reading 0 ... 2444945  =      0.000 ...   611.236 secs...
ecog
Reading 0 ... 2444945  =      0.000 ...   611.236 secs...
acc
emg
ecg


In [46]:
# Aretefact Removal

importlib.reload(preproc)
data_clean = {}
ch_nms_clean = {}
save_dir = os.path.join(
    runInfo0.fig_path,
    'preprocessing',
    runInfo0.store_str,
    runInfo0.preproc_sett,
    )
# save = None
for group in groups:
    data_clean[group], ch_nms_clean[group] = artefacts.artefact_selection(
        bids_dict=data_raw,  # raw BIDS group to process
        group=group,
        win_len=getattr(settings, group).win_len,
        n_stds_cut=getattr(settings, group).artfct_sd_tresh,  # number of std-dev from mean that is used as cut-off
        # to save: give directory, to show inline: give 'show', w/o fig: None
        save=None,  # if None: no figure saved
    )

START ARTEFACT REMOVAL: lfp_left
Ch 1: 0.0% is NaN (artefact or zero)
Ch 2: 0.0% is NaN (artefact or zero)
Ch 3: 0.0% is NaN (artefact or zero)
Ch 4: 0.0% is NaN (artefact or zero)
Ch 5: 0.0% is NaN (artefact or zero)
Ch 6: 0.0% is NaN (artefact or zero)
Ch 7: 0.16% is NaN (artefact or zero)
Ch 8: 0.16% is NaN (artefact or zero)
Ch 9: 0.0% is NaN (artefact or zero)
Ch 10: 0.0% is NaN (artefact or zero)
Ch 11: 0.0% is NaN (artefact or zero)
Ch 12: 0.0% is NaN (artefact or zero)
Ch 13: 0.0% is NaN (artefact or zero)
Ch 14: 0.0% is NaN (artefact or zero)
Ch 15: 0.0% is NaN (artefact or zero)
START ARTEFACT REMOVAL: lfp_right
Ch 1: 0.65% is NaN (artefact or zero)
Ch 2: 100.0% is NaN (artefact or zero)
Ch 3: 23.08% is NaN (artefact or zero)
Ch 4: 0.33% is NaN (artefact or zero)
Ch 5: 5.56% is NaN (artefact or zero)
Ch 6: 100.0% is NaN (artefact or zero)
Ch 7: 100.0% is NaN (artefact or zero)
Ch 8: 0.65% is NaN (artefact or zero)
Ch 9: 0.0% is NaN (artefact or zero)
Ch 10: 0.0% is NaN (artef

### 3. Bandpass Filtering

In [101]:
importlib.reload(fltrs)

data_bp = {}
for group in groups:
    data_bp[group] = fltrs.bp_filter(
        clean_dict=data_clean,
        group=group,
        sfreq=getattr(settings, group).Fs_orig,
        l_freq=getattr(settings, group).bandpass_f[0],
        h_freq=getattr(settings, group).bandpass_f[1],
        method='iir',  # faster than fir
    )

### 4. Notch-filtering for Powerline Noise

In [102]:
# notch filtering in BLOCKS

importlib.reload(fltrs)
save_dir = os.path.join(
    figpath,
    'preprocessing',
    runInfo0.store_str,
    getattr(settings, group).settings_f_name,
)
data_nf = {}
for group in data_bp.keys():
    print(f'Start Notch-Filter GROUP: {group}')
    data_nf[group] = fltrs.notch_filter(
        bp_dict=data_bp,  # NOT DICT ANYMORE MORE BUT NAMEDTUPLE!!!
        group=group,  # CALLS getattr() INSIDE FUNCTION WITH GROUP; CHANGE TO getattr() HERE??
        transBW=getattr(settings, group).transBW,  # based on figures for now: 10
        notchW=getattr(settings, group).notchW,  # based on figures for now: 2
        method='fir',  #iir (8th or. Butterwidth) takes too long
        ch_names=ch_nms_clean,
        save=None,  # if None: no figures made and saved
        verbose=False,
    )

Start Notch-Filter GROUP: lfp_left
Start Notch-Filter GROUP: lfp_right
Start Notch-Filter GROUP: ecog


### 5. Resampling


Since freq's of interest are up to +/- 100 - 120 Hz, according to the Nyquist-theorem the max sample freq does not need to be more than double (~ 250 Hz).

Check differences with resampling to 400 or 800 Hz later. Or working with wider windows.
- Swann '16: 800 Hz
- Heger/ Herff: 600 Hz (https://www.csl.uni-bremen.de/cms/images/documents/publications/IS2015_brain2text.pdf)


In [92]:
importlib.reload(preproc)

# resampling one run at a time
data_rs = {}  # dict to store resampled data
for group in groups:
    data_rs[group] = preproc.resample(
        data=data_nf,
        group=group,
        Fs_orig=getattr(settings, 'ecog').Fs_orig,
        Fs_new = getattr(settings, 'ecog').Fs_resample,
    )

### 6. Rereferencing



Common Practice LFP Re-referencing: difference between two nieghbouring contacts
- For segmented Leads: average every level


Relevant ECOG-rereferencing literature used: 
- Common Average Rereferencing (Liu ea, J Neural Eng 2015 (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5485665/)
- ECOG is local sign with spread +/- 3mm (Dubey, J Neurosc 2019): https://www.jneurosci.org/content/39/22/4299 
- READ ON - DATA ANALYSIS: Relevance of data-driven spatial filtering for invasive EEG. For gamma: CAR is probably sufficient. For alpha-beta: ... Hihg inter-subject variability in ECOG. (Shaworonko & Voytek, PLOS Comp Biol 2021: https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1009298)
- Submilimeter (micro)ECOG: http://iebl.ucsd.edu/sites/iebl.ucsd.edu/files/2018-06/Sub-millimeter%20ECoG%20pitch%20in%20human%20enables%20higher%20%EF%AC%81delity%20cognitiveneural%20state%20estimation.pdf


Check rereferencing methods:
- de Cheveigne/Arzounian NeuroImage 2018
- pre-prints Merk 2021 and Petersen 2021 (AG Kühn / AG Neumann)
- pre-print epilepsy ecog movement (MUMC)


P.M. Check further in to Spatial Filtering:
- Spatial filter estimation via spatio-spectral decomposition: ............ TO READ   (Nikulin & Curio, NeuroImage 2011, https://www.sciencedirect.com/science/article/pii/S1053811911000930?via%3Dihub)
- Spatio-Spectral Decomposition: proposed dimensionality-reduction instead of PCA (Haufe, ..., Nikulin, https://www.sciencedirect.com/science/article/pii/S1053811914005503?via%3Dihub)
- Also check: SPoC (Castano et al NeuroImage Clin 2020)


In [50]:
importlib.reload(reref)
data_rrf = {}
names = {}
for group in groups:
    data_rrf[group], names[group] = reref.rereferencing(
        data=data_rs,
        group=group,
        ch_names_og=ch_names,
        ch_names_clean=ch_nms_clean,
    )



======= REREFERENCING OVERVIEW ======

For lfp_left: Neigbouring Reref
Sec check: No time row present in orig names
Assume BS Vercise Cartesia X
Level ('L', 0) contains rows 0:3, or: ['LFP_L_1_', 'LFP_L_2_', 'LFP_L_3_']
Level ('L', 1) contains rows 3:6, or: ['LFP_L_4_', 'LFP_L_5_', 'LFP_L_6_']
Level ('L', 2) contains rows 6:9, or: ['LFP_L_7_', 'LFP_L_8_', 'LFP_L_9_']
Level ('L', 3) contains rows 9:12, or: ['LFP_L_10_', 'LFP_L_11_', 'LFP_L_12_']
Level ('L', 4) contains rows 12:15, or: ['LFP_L_13_', 'LFP_L_14_', 'LFP_L_15_']
Level ('L', 5) contains rows 15:15, or: []

 Auto Cleaning:
 In lfp_left: row 5 (LFP_L_4_5) only contained NaNs and is deleted


======= REREFERENCING OVERVIEW ======

For lfp_right: Neigbouring Reref
Sec check: No time row present in orig names
Assume BS Vercise Cartesia X
Level ('R', 0) contains rows 0:2, or: ['LFP_R_1_', 'LFP_R_3_']
Level ('R', 1) contains rows 2:4, or: ['LFP_R_4_', 'LFP_R_5_']
Level ('R', 2) contains rows 4:6, or: ['LFP_R_8_', 'LFP_R_9_']
Level

/Users/jeroenhabets/Research/CHARITE/projects/dyskinesia_neurophys/code/py_neuromodulation/dyskinesia/reref.py:98: RuntimeWarning: Mean of empty slice
  newdata[:, l + 1, :] = np.nanmean(


### 7. Saving Preprocessed Signals

In [85]:
importlib.reload(preproc)
for g in groups:
    preproc.save_arrays(data=data_rrf, group=g, runInfo=runInfo0)

In [57]:
# Load npy-files again
temp = np.load(os.path.join(
    f_dir, f_name))